In [6]:
import h5py
import numpy as np
from corsikaio import CorsikaParticleFile
import glob
from particle import Particle
from tqdm import tqdm as tqdm
from pathlib import Path

In [7]:
energy = 'spectrum_-2.7'
theta = 30

In [8]:
base_path = Path("/dicos_ui_home/antonpr/work/corsika_runs")
base_path = base_path / "34_simple_run"
path_to_res = base_path/"results"

corsika_datafiles = []
for file in path_to_res.glob("DAT*[!.long]"):
    corsika_datafiles.append(file)

corsika_datafiles = sorted(corsika_datafiles)
fnames = [int(f.name[-3:]) for f in corsika_datafiles]

iprev = 0
ss = 0
for i in fnames[1:]:
    if i > iprev + 1:
        ss += (i - iprev - 1)
        print(iprev, i, i - iprev - 1, ss)
    iprev = i

print(len(fnames), ss, len(fnames) + ss) 
# print(len(corsika_datafiles))


600 0 600


In [9]:
projectiles = [3122, 2212, 2112, 130, 321, 211]
products = [-11, 11, -12, 12, -13, 13, -14, 14, -16, 16,
            22, 130, -211, 211, -321, 321, -2112, 2112, -2212, 2212,
            -3122, 3122, 310]

# Get particle masses
mass_dict_GeV = {}

for projectile in np.append(projectiles, products):
    try:
        mass_dict_GeV[projectile] = Particle.from_pdgid(projectile).mass/1e3
    except TypeError:
        mass_dict_GeV[projectile] = 0.
        
corsika_id_dict = {13: 6, -13: 5, 
                   211: 8, -211: 9, 
                   2212: 14, -2212: 15,
                   2112: 13, -2112: 25,
                   12: 66, -12: 67,
                   14: 68, -14: 69,
                   321: 11, -321: 12}

In [10]:
### EXTRACT LEPTONS

get_pdg = [12, 13, 14, -12, -13, -14]

### EXTRACT HADRONS
#get_pdg = [211, -211, 2212, -2212, 2112, -2112]

In [11]:
num_levels = 4
level_range = np.arange(1, num_levels + 1, 1)

# Placeholders to store CORSIKA data per level: theta [rad], energy [GeV], and PDG ID
all_thetas_corsika = {}
all_energies_corsika = {}
all_unique_pids = {}


for pdg in get_pdg:

    all_thetas_corsika[pdg] = {}
    all_energies_corsika[pdg] = {}
    all_unique_pids[pdg] = {}

    for level in level_range:

        all_thetas_corsika[pdg][level] = []
        all_energies_corsika[pdg][level] = []
        all_unique_pids[pdg][level] = []

In [12]:
import time
import signal

def handler(signum, frame):
    print("Forever is over!")
    raise Exception("end of time")

signal.signal(signal.SIGALRM, handler)
#signal.alarm(3)

<Handlers.SIG_DFL: 0>

In [16]:
for filename in tqdm(corsika_datafiles, total = len(corsika_datafiles)):
    with CorsikaParticleFile(filename) as f:   
        
        counter = 0
        for e in f:
                    
            particle_data = e.particles

            particles_in_subblock = particle_data['particle_description']
            pids_in_subblock = (particles_in_subblock - (particles_in_subblock % 1000)) / 1000
            
            for pdg in get_pdg:
                
                show_corsika_id = corsika_id_dict[pdg]
                show_corsika_id_bar = corsika_id_dict[-pdg]
                
                part_mass = mass_dict_GeV[np.abs(pdg)]

                filter_pid_inds = np.where((pids_in_subblock == show_corsika_id))[0]

                filtered_obslevels = np.array(particles_in_subblock % 10)[filter_pid_inds] 
                
                filtered_pz = particle_data['pz'][filter_pid_inds]
                filtered_px = particle_data['px'][filter_pid_inds]
                filtered_py = particle_data['py'][filter_pid_inds]
                
                filtered_x = particle_data['x'][filter_pid_inds]
                filtered_y = particle_data['y'][filter_pid_inds]
                
                filtered_unique_ids = particles_in_subblock[filter_pid_inds]
                
                
                r = np.sqrt(filtered_x**2 + filtered_y**2)
                

                filtered_pT = np.sqrt(filtered_px**2 + filtered_py**2)
                
                filtered_ptot = np.sqrt(filtered_px**2 + filtered_py**2 + filtered_pz**2)
            

                filtered_theta = np.arctan2(filtered_pT, filtered_pz)
                filtered_phi = np.arctan2(filtered_py, filtered_px)
                
                init_dir = np.array([np.ones_like(filtered_phi) * np.sin(np.radians(theta)), 
                        np.zeros_like(filtered_phi) * np.sin(np.radians(theta)),
                        np.ones_like(filtered_phi) * np.cos(np.radians(theta))]).T
                
                final_dir = np.array([np.cos(filtered_phi) * np.sin(filtered_theta),
                                    np.sin(filtered_phi) * np.sin(filtered_theta),
                                    np.cos(filtered_theta)]).T
                
                dot_products = np.arccos(np.sum(init_dir * final_dir, axis=1))
                
                filtered_energies = np.sqrt(filtered_pT**2 + filtered_pz**2 + part_mass**2) - part_mass

                for obslev_id in level_range:

                    this_obslev_inds = np.where(filtered_obslevels == obslev_id)[0]

                    all_thetas_corsika[pdg][obslev_id].extend(dot_products[this_obslev_inds])

                    all_energies_corsika[pdg][obslev_id].extend(filtered_energies[this_obslev_inds])
                    
                    all_unique_pids[pdg][obslev_id].extend(filtered_unique_ids[this_obslev_inds])
                     

 19%|█▉        | 114/600 [07:28<31:50,  3.93s/it]


KeyboardInterrupt: 

In [ ]:
corsika_results_dict = {}

for pdg in get_pdg:
    corsika_results_dict[pdg] = {}
    for obslev_id in level_range:
        corsika_results_dict[pdg][obslev_id] = {'theta [rad]':[], 'energy [GeV]':[]}
        
        thetas = all_thetas_corsika[pdg][obslev_id]
        energies = all_energies_corsika[pdg][obslev_id]
        ids = all_unique_pids[pdg][obslev_id]
    
        corsika_results_dict[pdg][obslev_id]['theta [rad]'] = np.array(thetas)
        corsika_results_dict[pdg][obslev_id]['energy [GeV]'] = np.array(energies)
        corsika_results_dict[pdg][obslev_id]['pid'] = np.array(ids)
    

In [ ]:
corsika_results_dict['num_primaries'] = np.int64(len(corsika_datafiles)*500)

In [ ]:
def save_dict_to_hdf5(dic, filename):
    """
    ....
    """
    with h5py.File(filename, 'w') as h5file:
        recursively_save_dict_contents_to_group(h5file, '/', dic)

def recursively_save_dict_contents_to_group(h5file, path, dic):
    """
    ....
    """
    for key, item in dic.items():
        
        key = str(key)
        if isinstance(item, (np.ndarray, np.int64, np.float64, str, bytes)):
            h5file[path + key] = item
        elif isinstance(item, dict):
            recursively_save_dict_contents_to_group(h5file, path + key + '/', item)
        else:
            raise ValueError('Cannot save %s type'%type(item))

In [ ]:
dir_path = base_path
h5file_path = dir_path/'corsika_leptons_01.h5'

In [ ]:
save_dict_to_hdf5(corsika_results_dict, h5file_path)